In [1]:
using LaTeXStrings
using JLD2
using LinearAlgebra
using Statistics
using DifferentialEquations
using Random
using Folds
BLAS.set_num_threads(1)

In [140]:
function sampleSpinZPlus(n)
    θ = fill(acos(1 / sqrt(3)), n)
    ϕ = 2π * rand(n)                  
    return θ, ϕ
end
function sampleSpinZMinus(n)
    θ = fill(π - acos(1 / sqrt(3)), n)   
    ϕ = 2π * rand(n)                  
    return θ, ϕ
end


sampleSpinZMinus (generic function with 1 method)

In [141]:
function prob_func(prob, i, repeat)
    u0 = Vector{Float64}(undef, 2 * nAtoms)
    excited_indices_set = Set(excited_indices)
    non_excited_indices = setdiff(1:nAtoms, excited_indices)
    θ_excited, ϕ_excited = sampleSpinZPlus(length(excited_indices))
    u0[excited_indices] = θ_excited
    u0[nAtoms.+excited_indices] = ϕ_excited
    θ_non_excited, ϕ_non_excited = sampleSpinZMinus(length(non_excited_indices))
    u0[non_excited_indices] = θ_non_excited
    u0[nAtoms.+non_excited_indices] = ϕ_non_excited
    return remake(prob, u0=u0)
end


prob_func (generic function with 1 method)

In [142]:
function drift!(du, u, p, t)
    neighbors = get_neighbors_vectorized(nAtoms)
    Ω, Δ, V, Γ, γ = p
    θ = u[1:nAtoms]
    ϕ = u[nAtoms.+(1:nAtoms)]
    sqrt_3 = sqrt(3)
    dϕ_drift_sum = zeros(nAtoms)
    if case == 1
        dϕ_drift_sum[2:end-1] .= 2 .+ sqrt_3 .* (cos.(θ[1:end-2]) .+ cos.(θ[3:end]))
        dϕ_drift_sum[1] = 1 + sqrt_3 * cos(θ[2]) 
        dϕ_drift_sum[end] = 1 + sqrt_3 * cos(θ[end-1])
    end
    if case == 2
        for n in 1:nAtoms
            neighbor_indices = neighbors[n]
            dϕ_drift_sum[n] = sum(1 .+ sqrt_3 * cos.(θ[neighbor_indices]))
        end
    end
    cotθ = cot.(θ)
    cscθ = csc.(θ)
    dθ_drift = -2 .* Ω .* sin.(ϕ) .+ Γ .* (cotθ .+ cscθ ./ sqrt_3)
    dϕ_drift = -2 .* Ω .* cotθ .* cos.(ϕ) .+ (V / 2) .* dϕ_drift_sum .- Δ
    du[1:nAtoms] .= dθ_drift
    du[nAtoms.+(1:nAtoms)] .= dϕ_drift
end
function diffusion!(du, u, p, t)
    Ω, Δ, V, Γ, γ = p
    θ = u[1:nAtoms]
    sqrt_3 = sqrt(3)
    term1 = 9 / 6
    term2 = (4 * sqrt_3 / 6) .* cos.(θ)
    term3 = (3 / 6) .* cos.(2 .* θ)
    cscθ2 = csc.(θ) .^ 2
    diffusion = sqrt.(Γ .* (term1 .+ term2 .+ term3) .* cscθ2 .+ 4 .* γ)
    du[1:nAtoms] .= 0.0
    du[nAtoms.+(1:nAtoms)] .= diffusion
end


diffusion! (generic function with 1 method)

In [143]:
function get_neighbors_vectorized(nAtoms)
    matrix_size = sqrt(nAtoms) |> Int
    rows = [(div(i - 1, matrix_size) + 1) for i in 1:nAtoms]
    cols = [(mod(i - 1, matrix_size) + 1) for i in 1:nAtoms]
    neighbor_offsets = [
        (-1, 0),  # Up
        (1, 0),   # Down
        (0, -1),  # Left
        (0, 1)    # Right
    ]
    neighbors = Vector{Vector{Int}}(undef, nAtoms)
    for i in 1:nAtoms
        row, col = rows[i], cols[i]
        atom_neighbors = [
            (row + dr - 1) * matrix_size + (col + dc)
            for (dr, dc) in neighbor_offsets
            if 1 <= row + dr <= matrix_size && 1 <= col + dc <= matrix_size
        ]
        neighbors[i] = atom_neighbors
    end
    return neighbors
end


get_neighbors_vectorized (generic function with 1 method)

In [144]:
function computeTWA(nAtoms, tf, nT, nTraj, dt, Ω, Δ, V, Γ, γ)
    tspan = (0, tf)
    tSave = LinRange(0, tf, nT)
    u0 = (2 * nAtoms)
    p = (Ω, Δ, V, Γ, γ, nAtoms)

    prob = SDEProblem(drift!, diffusion!, u0, tspan, p)
    ensemble_prob = EnsembleProblem(prob; prob_func=prob_func)
    
    sol = solve(ensemble_prob, EM(), EnsembleThreads();
        saveat=tSave, trajectories=nTraj, maxiters=1e+7, dt=dt,
        abstol=1e-5, reltol=1e-5)
    
    return tSave, sol
end


computeTWA (generic function with 1 method)

In [145]:
function compute_spin_Sz(sol, nAtoms)
    θ = sol[1:nAtoms, :, :]
    Szs = sqrt(3) * cos.(θ)
    return Szs
end
global num_excited = Int(round(percent_excited * nAtoms))
global excited_indices = sort(randperm(nAtoms)[1:num_excited])

4-element Vector{Int64}:
 1
 2
 3
 4

In [152]:
Γ = 1
γ = 1 * Γ
Δ = 400 * Γ
V = Δ
natoms = [4,9,16]
tf = 25
nT = 1000
nTraj = 1
dt = 1e-4
percent_excited = 1.0
beta = 0.584
delta = 0.451
Ω_values = [0.1, 1, 2];

In [155]:
@time begin
    for nAtoms1 in natoms
        global nAtoms = nAtoms1
        for Ω1 in Ω_values
            Ω = Ω1
            @time t, sol = computeTWA(nAtoms, tf, nT, nTraj, dt, Ω, Δ, V, Γ, γ)
        end
    end
end

  2.268213 seconds (37.75 M allocations: 1.379 GiB, 12.36% gc time)
  2.286071 seconds (37.75 M allocations: 1.379 GiB, 11.47% gc time)
  2.324588 seconds (37.75 M allocations: 1.379 GiB, 11.51% gc time)
  3.215584 seconds (56.50 M allocations: 2.176 GiB, 12.51% gc time)
  3.134935 seconds (56.50 M allocations: 2.176 GiB, 11.82% gc time)
  3.344457 seconds (56.50 M allocations: 2.176 GiB, 12.10% gc time)
  4.662951 seconds (82.75 M allocations: 3.383 GiB, 14.30% gc time)
  4.602817 seconds (82.75 M allocations: 3.383 GiB, 13.97% gc time)
  4.704061 seconds (82.75 M allocations: 3.383 GiB, 14.20% gc time)
 30.544997 seconds (531.02 M allocations: 20.812 GiB, 12.98% gc time)
